In [3]:
template = """
FROM prolocutor/docker-python-sqlite:sqlite3.12

RUN cd /tmp/sqlite-autoconf-3120200 \                                                                    
        && ./configure --prefix=/usr --disable-static \                                                  
                CFLAGS="-g {{ for flag in flags }} {{ flag }} {{ endfor }}" \                                                                            
        && make \                                                                                        
        && make install \                                                                                
        && rm -r /tmp/*                                                                                  
                                                                                                         
RUN find /usr/local -depth \                                                                             
                \( \                                                                                     
                    \( -type d -a -name test -o -name tests \) \
                    -o \
                    \( -type f -a -name '*.pyc' -o -name '*.pyo' \) \
                \) -exec rm -rf '{}' + \
        && runDeps="$( \
                scanelf --needed --nobanner --recursive /usr/local \
                        | awk '{ gsub(/,/, "\nso:", $2); print "so:" $2 }' \
                        | sort -u \
                        | xargs -r apk info --installed \
                        | sort -u \
        )" \
        && apk add --virtual .python-rundeps $runDeps \
        && apk del .build-deps \
        && rm -rf /usr/src/python ~/.cache

CMD ["python3"]

"""
with open("Dockerfile.jinja2","w") as f:
    f.write(template)

In [46]:
trunk = {"Os":"-Os",
         "O2":"-O2"}

available_flags = {"fts4":"-DSQLITE_ENABLE_FTS4",
                   "rtree":"-DSQLITE_ENABLE_RTREE",
                   "column-metadata":"-DSQLITE_ENABLE_COLUMN_METADATA"}

tree = []

import os
from itertools import permutations
root = "./sqlite3.12/"

length = len(available_flags)

dockerfile_dir = []
for p in permutations(available_flags):
    for item in trunk:
        d = os.path.join(item,"/".join(p))
        try: os.makedirs(d)
        except: pass
            
        dockerfile_dir.append(d)
dockerfile_dir

['Os/column-metadata/rtree/fts4',
 'O2/column-metadata/rtree/fts4',
 'Os/column-metadata/fts4/rtree',
 'O2/column-metadata/fts4/rtree',
 'Os/rtree/column-metadata/fts4',
 'O2/rtree/column-metadata/fts4',
 'Os/rtree/fts4/column-metadata',
 'O2/rtree/fts4/column-metadata',
 'Os/fts4/column-metadata/rtree',
 'O2/fts4/column-metadata/rtree',
 'Os/fts4/rtree/column-metadata',
 'O2/fts4/rtree/column-metadata']

In [48]:

flag_combinations = []

for L in range(0, len(available_flags)+1):
    for subset in itertools.combinations(available_flags, L):
        for item in trunk:
            flag_combinations.append(sorted(subset+(item,)))
            
flag_combinations

[['Os'],
 ['O2'],
 ['Os', 'column-metadata'],
 ['O2', 'column-metadata'],
 ['Os', 'rtree'],
 ['O2', 'rtree'],
 ['Os', 'fts4'],
 ['O2', 'fts4'],
 ['Os', 'column-metadata', 'rtree'],
 ['O2', 'column-metadata', 'rtree'],
 ['Os', 'column-metadata', 'fts4'],
 ['O2', 'column-metadata', 'fts4'],
 ['Os', 'fts4', 'rtree'],
 ['O2', 'fts4', 'rtree'],
 ['Os', 'column-metadata', 'fts4', 'rtree'],
 ['O2', 'column-metadata', 'fts4', 'rtree']]

In [ ]:
from jinja2 import Template, Environment, FileSystemLoader
#Everything is a template
obreros_env = Environment(loader=FileSystemLoader('.'))
for t in templates:
    obreros_template = obreros_env.get_template('Dockerfile.jinja2').render(flags=flags)
    with open(t['file'], "w") as f:
        f.write(obreros_template)